In [2]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [3]:
spacex_df.loc[(spacex_df['Payload Mass (kg)'] >= 500) & (spacex_df['Payload Mass (kg)'] <= 600)]

,Unnamed: 0,Flight Number,Launch Site,Mission Outcome,class,Payload Mass (kg),Booster Version,Booster Version Category
2,2,3,CCAFS LC-40,Success,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,Success,0,500.0,F9 v1.0 B0006,v1.0
13,13,15,CCAFS LC-40,Success,0,570.0,F9 v1.1 B1013,v1.1
26,26,6,VAFB SLC-4E,Success,0,500.0,F9 v1.1 B1003,v1.1
27,27,21,VAFB SLC-4E,Success,0,553.0,F9 v1.1 B1017,v1.1


In [4]:
spacex_df[spacex_df['Launch Site']=='VAFB SLC-4E']

,Unnamed: 0,Flight Number,Launch Site,Mission Outcome,class,Payload Mass (kg),Booster Version,Booster Version Category
26,26,6,VAFB SLC-4E,Success,0,500.0,F9 v1.1 B1003,v1.1
27,27,21,VAFB SLC-4E,Success,0,553.0,F9 v1.1 B1017,v1.1
28,28,29,VAFB SLC-4E,Success,1,9600.0,F9 FT B1029.1,FT
29,29,37,VAFB SLC-4E,Success,1,9600.0,F9 FT B1036.1,FT
30,30,40,VAFB SLC-4E,Success,1,475.0,F9 FT B1038.1,FT
31,31,42,VAFB SLC-4E,Success,1,9600.0,F9 B4 B1041.1,B4
32,32,46,VAFB SLC-4E,Success,0,9600.0,F9 FT B1036.2,FT
33,33,49,VAFB SLC-4E,Success,0,2150.0,F9 FT B1038.2,FT
34,34,51,VAFB SLC-4E,Success,0,9600.0,F9 B4 B1041.2,B4
35,35,55,VAFB SLC-4E,Success,0,6460.0,F9 B4 B1043.2,B4


In [ ]:


# Create a dash application
app = dash.Dash(__name__)
sites=[{'label': i, 'value': i} for i in spacex_df['Launch Site'].unique()]
all_site=[{'label': 'All Sites', 'value': 'ALL'},]
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                  dcc.Dropdown(id='site-dropdown',
                                                options=all_site+sites,
                                                value='ALL',
                                                placeholder="Select a Launch Site here",
                                                searchable=True
                                                ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                    dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0',2000: '2000',4000: '4000',6000: '6000',8000: '8000',10000: '10000'},
                                                value=[min_payload,max_payload]),
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, values='class', names='Launch Site', title='Total Success Launches By Site')
        return fig
    else:
        fig = px.pie(spacex_df[spacex_df['Launch Site']==entered_site], 
        names='class', 
        title='Total success Launches for site '+entered_site)
        return fig
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'), Input(component_id="payload-slider", component_property="value")])
def get_scatter_chart(entered_site,Load):
    Load_min,Load_max=Load
    filtered_df = spacex_df.loc[(spacex_df['Payload Mass (kg)'] >= Load_min) & (spacex_df['Payload Mass (kg)'] <= Load_max)]

    if entered_site == 'ALL':
        fig = px.scatter(filtered_df, x="Payload Mass (kg)", y="class",color="Booster Version Category")
        return fig
    else:
        fig = px.scatter(filtered_df[filtered_df['Launch Site']==entered_site], x="Payload Mass (kg)", y="class",color="Booster Version Category")

        return fig

# Run the app
if __name__ == '__main__':
    app.run_server()


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [08/Mar/2022 22:56:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2022 22:56:19] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_9_1m1617985068.8.7.min.js HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2022 22:56:19] "GET /_dash-component-suites/dash_renderer/react@16.v1_9_1m1617985068.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2022 22:56:19] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_9_1m1617985068.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2022 22:56:19] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_9_1m1617985068.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2022 22:56:19] "GET /_dash-component-suites/dash_core_components/dash_core_components-shared.v1_16_0m1617985098.js HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2022 22:56:19] "GET /_dash-component-suites/dash_html_components/dash_html_components.v1_1_3m1617985573.min.js HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2022 22